In [1]:
using Pkg
# Pkg.activate(pwd())
Pkg.DEFAULT_IO[] = stdout
Pkg.status()

     Project demo v0.1.0
      Status `~/Documents/julia/demo/Project.toml`
  [336ed68f] CSV v0.9.6
  [324d7699] CategoricalArrays v0.10.2
  [7876af07] Example v0.5.4 `~/.julia/dev/Example`
  [ced58d7a] HighDimMixedModels v0.1.0 `~/.julia/dev/HighDimMixedModels`
  [a1dec852] Metida v0.12.4
  [ff71e718] MixedModels v4.6.0
  [14b8a8f1] PkgTemplates v0.7.19
  [bd369af6] Tables v1.6.0
  [44d3d7a6] Weave v0.10.10


In [2]:
using LinearAlgebra
using DataFrames
using StableRNGs; rng = StableRNG(1);
using StatsModels
using MixedModels
using CSV, CategoricalArrays

In [3]:
using HighDimMixedModels

In [4]:
#MixedModels.datasets()
df = DataFrame(MixedModels.dataset(:cbpp))
df[!,:period] = map(x->parse(Float64,x),df[:,:period])
select!(df,[:period,:herd,:incid,:hsz])
first(df,5)

,period,herd,incid,hsz
,Float64,String,Int8,Int8
1,1.0,H01,2,14
2,2.0,H01,3,12
3,3.0,H01,4,9
4,4.0,H01,0,5
5,1.0,H02,3,22


In [5]:
f = @formula(period ~ (1|herd))
#f = @formula(period ~ herd + incid + hsz)
#fit(MixedModel, @formula(period ~ incid + hsz +(1|herd)), df, REML = true)

FormulaTerm
Response:
  period(unknown)
Predictors:
  (herd)->1 | herd

In [6]:
for i in 1:size(df,2)
    if(isa(df[1,i], Number))
        df[!,i] = convert(Vector{Float64},df[:,i])
    end
end

In [7]:
sch = schema(df)
form = apply_schema(f, sch, highDimMixedModel)

FormulaTerm
Response:
  period(continuous)
Predictors:
  1
  (1 | herd)

In [8]:
f

FormulaTerm
Response:
  period(unknown)
Predictors:
  (herd)->1 | herd

In [9]:
form

FormulaTerm
Response:
  period(continuous)
Predictors:
  1
  (1 | herd)

## High Dim Data

In [4]:
raw = DataFrame(CSV.File("raws.csv"))
raw = raw[1:30, 1:50]
first(raw,5)

,...1,...2,...3,...4,...5,...6,...7,...8,...9
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.5,0.5
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5
3,1.0,0.0,0.5,0.0,1.0,0.0,0.0,0.0,0.5
4,0.0,0.0,1.0,0.5,0.0,0.0,0.5,0.0,0.0
5,0.0,0.5,0.0,0.0,0.0,1.0,0.5,0.0,1.0


In [5]:
raw[!, :response] = rand(rng, 30)
raw[!, :trt] = rand(["A","B","C"], 30)
df = raw
first(df[:,45:52],5)

,...45,...46,...47,...48,...49,...50,response,trt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.0,0.0,0.0,0.0,0.0,0.0,0.585195,B
2,0.0,1.5,0.0,0.0,0.5,0.0,0.0773379,B
3,0.5,1.0,0.0,1.0,0.5,0.0,0.716628,C
4,0.5,1.0,0.0,0.0,0.5,0.0,0.320357,B
5,0.5,0.0,0.0,1.0,0.5,0.0,0.653093,B


In [12]:
for i in 1:size(df,2)
    if(isa(df[1,i], Number))
        df[!,i] = convert(Vector{Float64},df[:,i])
    end
end

In [13]:
M = highDimMat(Matrix(df[:,1:48]))
X = XMat(Matrix(df[:,49:50]))

XMat{Float64, Matrix{Float64}}([0.0 0.0; 0.5 0.0; … ; 2.0 0.0; 1.0 1.0])

In [14]:
# The function doesn't need formula to construct M and X, so the formula constucted here is only for extracting Z, 
# doesn't matter we have intercept or not
f = @formula(response ~ (1|trt))
sch = schema(df)
form = apply_schema(f, sch, highDimMixedModel)

FormulaTerm
Response:
  response(continuous)
Predictors:
  1
  (1 | trt)

In [15]:
name = "response"

"response"

In [16]:
typeof(f.lhs)

Term

In [26]:
typeof(form.rhs[2])

RandomEffectsTerm

In [32]:
CategoricalTerm(:trt)

LoadError: MethodError: no method matching CategoricalTerm(::Symbol)
[0mClosest candidates are:
[0m  CategoricalTerm(::Symbol, [91m::StatsModels.ContrastsMatrix{C, T, U} where U[39m) where {C, T} at /Users/zyxu/.julia/packages/StatsModels/m1jYD/src/terms.jl:237

In [28]:
typeof(term("trt"))

Term

In [17]:
y, pre = modelcols(form,df)
FixedEffectMatrix,ZMatrix = pre 
Z = HighDimMixedModels.ReMat(ZMatrix)

30×3 HighDimMixedModels.ReMat{Float64, MixedModels.ReMat{Float64, 1}}:
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  1.0  0.0
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 ⋮         
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  1.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 1.0  0.0  0.0
 0.0  0.0  1.0
 0.0  0.0  1.0
 1.0  0.0  0.0
 0.0  1.0  0.0
 0.0  1.0  0.0

In [18]:
first(df[:,45:52], 5)

,...45,...46,...47,...48,...49,...50,response,trt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,String
1,0.0,0.0,0.0,0.0,0.0,0.0,0.585195,C
2,0.0,1.5,0.0,0.0,0.5,0.0,0.0773379,A
3,0.5,1.0,0.0,1.0,0.5,0.0,0.716628,B
4,0.5,1.0,0.0,0.0,0.5,0.0,0.320357,C
5,0.5,0.0,0.0,1.0,0.5,0.0,0.653093,C


In [6]:
HMM = highDimMixedModel(df, "response", "trt", 1:48, 49:50)

response

LoadError: ArgumentError: blocking variables (those behind |) must be Categorical (:(term(nameOfReMat)) is not)